In [3]:
import pandas as pd
from sshtunnel import SSHTunnelForwarder
import pymysql
from config import ssh_username, ssh_host, ssh_password, ssh_port, mysql_database, mysql_host, mysql_password, mysql_port, mysql_username

# Establish SSH tunnel
with SSHTunnelForwarder(
    (ssh_host, ssh_port),
    ssh_username=ssh_username,
    ssh_password=ssh_password,
    remote_bind_address=(mysql_host, mysql_port)
) as tunnel:
    print(f'Tunnel local bind port: {tunnel.local_bind_port}')
    print(f'Tunnel is active: {tunnel.is_active}')

    # Connect to MySQL through the tunnel
    conn = pymysql.connect(
        host=mysql_host,
        port=tunnel.local_bind_port,
        user=mysql_username,
        password=mysql_password,
        database=mysql_database,
        connect_timeout=30,  # Increase the connection timeout
    )

    # target_db = 'opencartdb'

    # query = "SELECT * FROM oc_order"
    query_2 = "SELECT * FROM otto_authentication"
    # table_query = "SHOW TABLES"

    # data = pd.read_sql_query(query, conn)
    query_2 = pd.read_sql_query(query_2, conn)
    conn.close()
    

Tunnel local bind port: 50149
Tunnel is active: True
